In [1]:
import pandas as pd
import math

In [2]:
df1 = pd.read_csv("../data/animals.csv", sep=',', header=0)

In [3]:
print(df1.shape)
print(df1)


(20, 7)
   animal  war  fly  ver  end  gro  hai
0     ant    1    1    1  1.0  2.0    1
1     bee    1    2    1  1.0  2.0    2
2     cat    2    1    2  1.0  1.0    2
3     cpl    1    1    1  1.0  1.0    2
4     chi    2    1    2  2.0  2.0    2
5     cow    2    1    2  1.0  2.0    2
6     duc    2    2    2  1.0  2.0    1
7     eag    2    2    2  2.0  1.0    1
8     ele    2    1    2  2.0  2.0    1
9     fly    1    2    1  1.0  1.0    1
10    fro    1    1    2  2.0  NaN    1
11    her    1    1    2  1.0  2.0    1
12    lio    2    1    2  NaN  2.0    2
13    liz    1    1    2  1.0  1.0    1
14    lob    1    1    1  1.0  NaN    1
15    man    2    1    2  2.0  2.0    2
16    rab    2    1    2  1.0  2.0    2
17    sal    1    1    2  1.0  NaN    1
18    spi    1    1    1  NaN  1.0    2
19    wha    2    1    2  2.0  2.0    1


In [4]:
# Clean up the data 
# replace NaN with 0
df1 = df1.fillna(0)

# Normalization



In [20]:
#compute euclidean distance
def euclideanDist(obj1, obj2):
    '''
    input of equal length list obj1 and obj2
    returns square of pairwise euclidean distance for speeding up the algorithm
    '''
    
    dist = 0
    for i in range(1, len(obj1)):
        d = (obj1[i] -obj2[i])
        dist += d*d

    return(dist)

#compute UPGMA distance
def upgma(obj1, obj2):
    '''
    input : list obj1 and obj2
    returns square of pairwise euclidean distance for speeding up the algorithm
    '''
    
    dist = 0
    for i in range(0, len(obj1)):
        for j in range(0,len(obj2)):
            d = math.sqrt(euclideanDist(df1.iloc[obj1[i]].tolist(), df1.iloc[obj2[j]].tolist()))
            dist += d

    return(dist/(len(obj1) * len(obj2)))

In [23]:
upgma([15,4], [16,5])

1.0

In [30]:
# Agglomerative approach
result_cluster = []

# First pass

# Compute the distance between all data points
similarity_distance = []
obj1 = []
obj2 = []

for i_row in range(0, df1.shape[0]):
    distance = []
    obj1 = df1.iloc[i_row].tolist()

    for j_row in range(0, i_row+1):        
        obj2 = df1.iloc[j_row].tolist()
        distance.append(euclideanDist(obj1, obj2))

    similarity_distance.append(distance)
print(similarity_distance)

# first find the global minimum then use that to find the related pair 
min_dist = 9999999
min_dist_pair = []
for i in range(0, len(similarity_distance)):
    for j in range(0, len(similarity_distance[i])):
        if i != j:
            if similarity_distance[i][j] <= min_dist:
                min_dist = similarity_distance[i][j]

for i in range(0, len(similarity_distance)):
    for j in range(0, len(similarity_distance[i])):
        if i != j:
            if similarity_distance[i][j] <= min_dist:
                min_dist_pair.append([i,j])                

print(min_dist)
print(min_dist_pair)

result_cluster.append(min_dist_pair)

# This means all these individual pairs are clustered first but not all of them belongs to
# the single cluster
# I don't know how to handle this

# for the second pass 
#needed to compute the distance between the just clustered pair and then remaining single ones 

# as an idea:
# use the row number of the data set to form a cluster and store each row in
# a cluster as a seperate row and use that list to decide whether it is already 
# clustered cluster or just some row that has never been clustered before 

#second pass 
# compute the distance between the 1st cluster vs remaining points, to compute the distance
# between the cluster use UPGMA (unweighted pair group method using arithmatic mean)
clustered_list = []
for i in range(0,len(min_dist_pair)):
    for j in range(0, len(min_dist_pair[i])):
        clustered_list.append(min_dist_pair[i][j])

new_list = []
for i in range(0, len(min_dist_pair)):
    new_list.append(min_dist_pair[i])

for i in range(0, df1.shape[0]):
    if i not in clustered_list:
        new_list.append([i])

print(new_list)

# compute the similarity matrix:
similarity_distance = []
obj1 = []
obj2 = []

for i_row in range(0, len(new_list)):
    distance = []
    obj1 = new_list[i_row]

    for j_row in range(0, i_row+1):        
        obj2 = new_list[j_row]
        if len(obj1) == 1 and len(obj2) == 1:
            distance.append(euclideanDist(df1.iloc[obj1[0]].tolist(), df1.iloc[obj2[0]].tolist()))
        else:
            distance.append(upgma(obj1, obj2))

    similarity_distance.append(distance)
print(similarity_distance)

# find the minimum distance
# first find the global minimum then use that to find the related pair 
min_dist = 9999999
min_dist_pair = []
for i in range(0, len(similarity_distance)):
    for j in range(0, len(similarity_distance[i])):
        if i != j:
            if similarity_distance[i][j] <= min_dist:
                min_dist = similarity_distance[i][j]

for i in range(0, len(similarity_distance)):
    for j in range(0, len(similarity_distance[i])):
        if i != j:
            if similarity_distance[i][j] <= min_dist: 
                if new_list[i] in  min_dist_pair or new_list[j] in  min_dist_pair:
                    if new_list[i] in  min_dist_pair:
                        for i in range(0,len(min_dist_pair)):
                            if new_list[i] in  min_dist_pair[i]:
                                min_dist_pair[i].append(new_list[j])

                else:
                    min_dist_pair.append([new_list[i], new_list[j]]) 


print(min_dist)
print(min_dist_pair)

result_cluster.append(min_dist_pair)

# print (result_cluster)

[[0.0], [2.0, 0.0], [4.0, 4.0, 0.0], [2.0, 2.0, 2.0, 0.0], [4.0, 4.0, 2.0, 4.0, 0.0], [3.0, 3.0, 1.0, 3.0, 1.0, 0.0], [3.0, 3.0, 3.0, 5.0, 3.0, 2.0, 0.0], [5.0, 5.0, 3.0, 5.0, 3.0, 4.0, 2.0, 0.0], [3.0, 5.0, 3.0, 5.0, 1.0, 2.0, 2.0, 2.0, 0.0], [2.0, 2.0, 4.0, 2.0, 6.0, 5.0, 3.0, 3.0, 5.0, 0.0], [6.0, 8.0, 4.0, 4.0, 6.0, 7.0, 7.0, 3.0, 5.0, 4.0, 0.0], [1.0, 3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 4.0, 2.0, 3.0, 5.0, 0.0], [4.0, 4.0, 2.0, 4.0, 4.0, 1.0, 3.0, 7.0, 5.0, 6.0, 10.0, 3.0, 0.0], [2.0, 4.0, 2.0, 2.0, 4.0, 3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 1.0, 4.0, 0.0], [4.0, 6.0, 4.0, 2.0, 8.0, 7.0, 7.0, 5.0, 7.0, 2.0, 2.0, 5.0, 8.0, 2.0, 0.0], [4.0, 4.0, 2.0, 4.0, 0.0, 1.0, 3.0, 3.0, 1.0, 6.0, 6.0, 3.0, 4.0, 4.0, 8.0, 0.0], [3.0, 3.0, 1.0, 3.0, 1.0, 0.0, 2.0, 4.0, 2.0, 5.0, 7.0, 2.0, 1.0, 3.0, 7.0, 1.0, 0.0], [5.0, 7.0, 3.0, 3.0, 7.0, 6.0, 6.0, 4.0, 6.0, 3.0, 1.0, 4.0, 7.0, 1.0, 1.0, 7.0, 6.0, 0.0], [3.0, 3.0, 3.0, 1.0, 7.0, 4.0, 6.0, 8.0, 8.0, 3.0, 7.0, 4.0, 3.0, 3.0, 3.0, 7.0, 4.0, 4.0, 0.0], [3.0, 5.0,